In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cols_to_keep = [
    "Source",
    "Severity",
    "Temperature(F)",
    "Humidity(%)",
    "Pressure(in)",
    "Visibility(mi)",
    "Wind_Speed(mph)",
    "Weather_Condition",
    "Start_Time",
]
raw_df = pd.read_csv("raw_data.csv", usecols=cols_to_keep)
raw_df

In [ ]:
df = raw_df.loc[raw_df["Source"] == "Source2",]
df = df.drop(["Source"], axis=1)
print("The shape of data is:", (df.shape))

In [ ]:
df = df.dropna(subset=['Humidity(%)', "Temperature(F)", "Severity", "Weather_Condition"])

df["Temperature(C)"] = (df["Temperature(F)"] - 32) * 5 / 9

df = df[(df['Humidity(%)'] >= 0) & (df['Humidity(%)'] <= 100)] 
df = df[(df["Temperature(C)"] > -35) & (df["Temperature(C)"] < 45)]

df["is_Rain"] = df["Weather_Condition"].str.contains("Rain|Storm|Shower|Thunder", case=False, na=False)
df["is_Snow"] = df["Weather_Condition"].str.contains("Snow|Sleet|Ice|Blizzard|Squalls|Pellets", case=False, na=False)
df["is_Fog"] = df["Weather_Condition"].str.contains("Fog|Mist|Haze", case=False, na=False)
df["is_Clear"] = df["Weather_Condition"].str.contains("Clear|Fair", case=False, na=False)
df["is_Cloud"] = df["Weather_Condition"].str.contains("Cloud|Overcast", case=False, na=False)

df = df.loc[df['Start_Time'] > "2019-03-10",:]
df = df.drop(['Start_Time'], axis=1)

df['HighSeverity'] = 0
df.loc[df['Severity'].isin([4]), 'HighSeverity'] = 1

print(df.HighSeverity.value_counts())

cols_to_export = [
    'Severity', # NECESARIA para el color (1-4) en D3
    'Humidity(%)', 
    'Temperature(C)', 
    'Weather_Condition', 
    'is_Rain', 'is_Snow', 'is_Fog', 'is_Clear', 'is_Cloud', 
    'HighSeverity' # Para otros análisis
]

df_sample = df[cols_to_export].sample(n=10000, random_state=42)

df_sample.to_csv("data.csv", index=False)
print(f"Archivo exportado con {len(df_sample)} registros.")
print(df_sample.head())